## This notebook has the following topics -
1. Use langchain to run an LLM
2. Combines LLMs and Prompts in a Chain
3. Dynamically calling chains based on user inputs using tool, llm and agent
4. Adding state / memory to chains & agents
5. Building a chat app using Chat Prompts
6. Chat Prompt Templates
7. Chains with Chat Models
8. Chains with Chat Models using state / memory

### Building a language model application

In [1]:
import os
from langchain.llms import OpenAI

In [2]:
llm = OpenAI(temperature = 0.9)

text = 'What would be a good name for a company that makes colorful socks?'
print(llm(text))



Socktastic!


### Chains : Combine LLMs and prompts in a multi-step workflow
Convert the above user text into prompts

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

In [4]:
llm = OpenAI(temperature = 0.9)

prompt = PromptTemplate(
        input_variables = ["product"],
        template = 'What would be a good name for a company that makes {product}?',)

In [5]:
print(prompt.format(product = "colorful socks"))

What would be a good name for a company that makes colorful socks?


In [6]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt)

In [7]:
chain.run("colorful socks")

'\n\nSox in the Box.'

### Agents: Dynamically call chains based on User Inputs

Agents decide which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

* Tool : A function that performs a specific duty e.g. google search, database lookup, other chains etc. Input & Output : Both are string
* LLM : The language model powering the agent
* Agent : The agent to use

In [8]:
# Here we will use the SerpAPI for Google search

In [9]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

In [10]:
# Load llm to control the agent-
llm = OpenAI(temperature = 0)

In [13]:
# Load some tools -
tools = load_tools(['serpapi', 'llm-math'], llm = llm)

In [15]:
# Let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [17]:
# Testing the agent -
agent.run("What was the high temperature in Bangalore yesterday in Celcius ? What are some open problems in earth sciences ?")



> Entering new AgentExecutor chain...
 I need to find out the temperature in Bangalore and some open problems in earth sciences.
Action: Search
Action Input: "High temperature in Bangalore yesterday" and "Open problems in earth sciences"
Observation: 80 °F · 81 °F · 79 °F ...
Thought: I need to convert the temperature to Celcius
Action: Calculator
Action Input: 80 °F in Celcius
Observation: Answer: 26.666666666666668
Thought: I now know the final answer
Final Answer: The high temperature in Bangalore yesterday in Celcius was 26.666666666666668. Some open problems in earth sciences include climate change, ocean acidification, and land use change.

> Finished chain.


'The high temperature in Bangalore yesterday in Celcius was 26.666666666666668. Some open problems in earth sciences include climate change, ocean acidification, and land use change.'

### Memory : Add State to Chains & Agents
The chains and agents are stateless by default. But you can have a chain or agent with some concept of "memory" so that it remembers information about its previous interactions. 
The simple use case is when designing a chatbot, you need to remember the previos messages as a context like a "short-term memory" or sometimes "long-term memory"

In [18]:
from langchain import OpenAI, ConversationChain

In [19]:
llm = OpenAI(temperature = 0)
conversation = ConversationChain(llm = llm, verbose = True)
output = conversation.predict(input  = 'Hi there !')
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there !
AI:

> Finished chain.
 Hi there! How can I help you?


In [20]:
output = conversation.predict(input = "Just want to have a general converstaion with you. ")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there !
AI:  Hi there! How can I help you?
Human: Just want to have a general converstaion with you. 
AI:

> Finished chain.
 Sure! What would you like to talk about?


### Building a langugae model application : Chat Models
The above ConversationChain interface you have input, output as texts. But for a chat application, it can have a better abstaction like "chat" schema

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature = 0)

In [23]:
chat([HumanMessage(content = 'Translate this sentence from English to Hindi. I love programming')])

AIMessage(content='मुझे प्रोग्रामिंग करना पसंद है।', additional_kwargs={})

You can pass multiple messages for OpenAI's gpt-3.5-turbo and gpt-4 models

In [24]:
messages = [
    SystemMessage(content = "You are a helpful assistant that translates English to Hindi."),
    HumanMessage(content = "Translate this sentence from English to Hindi. I love singing Indian Classical"),
]

chat(messages)

AIMessage(content='मुझे भारतीय शास्त्रीय गायन करना पसंद है।', additional_kwargs={})

In [25]:
batch_messages = [
    [
        SystemMessage(content = "You are a helpful assistant that translates English to Hindi."),
        HumanMessage(content = "Translate this sentence from English to Hindi. I love programming.")
    ],
    [
        SystemMessage(content = "You are a helpful assistant that translates English to Hindi."),
        HumanMessage(content = "Translate this sentence from English to Hindi. I love Machine Learning.")
    ],
]

result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='मुझे प्रोग्रामिंग करना पसंद है।', generation_info=None, message=AIMessage(content='मुझे प्रोग्रामिंग करना पसंद है।', additional_kwargs={}))], [ChatGeneration(text='मुझे मशीन लर्निंग से प्यार है।', generation_info=None, message=AIMessage(content='मुझे मशीन लर्निंग से प्यार है।', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 73, 'completion_tokens': 62, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo'})

In [33]:
result.generations

[[ChatGeneration(text='मुझे प्रोग्रामिंग करना पसंद है।', generation_info=None, message=AIMessage(content='मुझे प्रोग्रामिंग करना पसंद है।', additional_kwargs={}))],
 [ChatGeneration(text='मुझे मशीन लर्निंग से प्यार है।', generation_info=None, message=AIMessage(content='मुझे मशीन लर्निंग से प्यार है।', additional_kwargs={}))]]

In [34]:
result.llm_output

{'token_usage': {'prompt_tokens': 73,
  'completion_tokens': 62,
  'total_tokens': 135},
 'model_name': 'gpt-3.5-turbo'}

### Chat Prompt Templates

In [40]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Hindi", text="I love programming.").to_messages())
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content='मुझे प्रोग्रामिंग से प्यार है।', additional_kwargs={})

### Chains with Chat Models

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [42]:
chat = ChatOpenAI(temperature = 0)
chat

ChatOpenAI(verbose=False, callback_manager=<langchain.callbacks.shared.SharedCallbackManager object at 0x1226cf880>, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_organization=None, request_timeout=60, max_retries=6, streaming=False, n=1, max_tokens=None)

In [43]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm = chat, prompt = chat_prompt)
chain.run(input_language = "English", output_language = "Hindi", text = "I love programming.")

'मुझे प्रोग्रामिंग से प्यार है।'

### Agents with Chat Models

In [44]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [45]:
# Load the model to control the agent -
chat = ChatOpenAI(temperature = 0)

In [46]:
# Load the tools 
llm = OpenAI(temperature = 0)
tools = load_tools(["serpapi", "llm-math"], llm = llm)

In [47]:
# Let's initialize an agent with the tools, the language model and the type of agent we want to use -
agent = initialize_agent(tools, chat, agent = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose = True)

In [50]:
# Now let's test it out -
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: I need to use a search engine to find Olivia Wilde's boyfriend and a calculator to raise his age to the 0.23 power.
Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde boyfriend"
}
```

Observation: Sudeikis and Wilde's relationship ended in November 2020. Wilde was publicly served with court documents regarding child custody while she was presenting Don't Worry Darling at CinemaCon 2022. In January 2021, Wilde began dating singer Harry Styles after meeting during the filming of Don't Worry Darling.
Thought:I need to use a search engine to find Harry Styles' current age.
Action:
```
{
  "action": "Search",
  "action_input": "Harry Styles age"
}
```

Observation: 29 years
Thought:Now I need to calculate 29 raised to the 0.23 power.
Action:
```
{
  "action": "Calculator",
  "action_input": "29^0.23"
}
```


Observation: Answer: 2.169459462491557
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-JnI81ckwmqzayKRM3GaHFrYA on requests per min. Limit: 3 / min. Please try again in 20s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-JnI81ckwmqzayKRM3GaHFrYA on requests per min. Limit: 3 / min. Please try again in 20s. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a pay

I now know the final answer.
Final Answer: 2.169459462491557

> Finished chain.


'2.169459462491557'

### Memory : Add State to Chains and Agents

In [51]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

In [52]:
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details \
                                               from its context. If the AI does not know the answer to a question, it truthfully says that it does not know."),
    MessagesPlaceholder(variable_name = "history"),
    HumanMessagePromptTemplate.from_template("{input}")

])

In [53]:
llm = ChatOpenAI(temperature = 0)
memory = ConversationBufferMemory(return_messages = True)
conversation = ConversationChain(memory = memory, prompt = prompt, llm = llm)

In [55]:
conversation.predict(input = "Hi there ! ")

'Hello! How can I assist you today?'

In [56]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

"That sounds like fun! I'm always here to chat. What would you like to talk about?"

In [57]:
conversation.predict(input="Tell me about yourself.")

"Sure! I'm an AI language model designed to assist with a variety of tasks, from answering questions to generating text. I'm powered by a large neural network that has been trained on vast amounts of data, allowing me to understand and respond to a wide range of topics. I'm constantly learning and improving, so I can provide more accurate and helpful responses over time. Is there anything specific you'd like to know about me?"